<a href="https://colab.research.google.com/github/siminikita/React-lessons/blob/main/project(data_analytics).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [3]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [4]:
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

In [5]:
df_list = []

In [6]:
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-07-31  196.059998  196.490005  195.259995  196.449997  196.185074   
       2023-08-01  196.240005  196.729996  195.279999  195.610001  195.346207   
       2023-08-02  195.039993  195.179993  191.850006  192.580002  192.320297   
       2023-08-03  191.570007  192.369995  190.690002  191.169998  190.912201   
       2023-08-04  185.520004  187.380005  181.919998  181.990005  181.744583   

                      Volume  
Ticker Date                   
AAPL   2023-07-31   38824100  
       2023-08-01   35175100  
       2023-08-02   50389300  
       2023-08-03   61235200  
       2023-08-04  115799700  


In [8]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-07-31  196.059998  196.490005  195.259995  196.449997   
1   AAPL 2023-08-01  196.240005  196.729996  195.279999  195.610001   
2   AAPL 2023-08-02  195.039993  195.179993  191.850006  192.580002   
3   AAPL 2023-08-03  191.570007  192.369995  190.690002  191.169998   
4   AAPL 2023-08-04  185.520004  187.380005  181.919998  181.990005   

    Adj Close     Volume  
0  196.185074   38824100  
1  195.346207   35175100  
2  192.320297   50389300  
3  190.912201   61235200  
4  181.744583  115799700  


In [10]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [11]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [12]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
59  177.080002  175.314502
60  176.585002  175.388502
61  175.715002  175.422002
62  174.333002  175.232002
63  173.270001  175.082502

[64 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
192         NaN         NaN
193         NaN         NaN
194         NaN         NaN
195         NaN         NaN
196         NaN         NaN
..          ...         ...
251  139.414999  136.950001
252  139.506999  137.483501
253  138.003999  137.244000
254  136.319000  136.759500
255  134.801000  136.337000

[64 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
..          ...         ...
123  330.182007  324.33

In [13]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [14]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [15]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()